In [1]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
iris_data = iris.data
iris_target = iris.target


iris_new_target = np.array([-1])
iris_new_data = [iris_data[0]]

for i in range(len(iris_target)-1):
    if iris_target[i] == 1:
        iris_new_target = np.append(iris_new_target,-1)
        iris_new_data = np.append(iris_new_data, [iris_data[i+1]],axis = 0)
    elif iris_target[i] == 2:
        iris_new_target = np.append(iris_new_target,1)
        iris_new_data = np.append(iris_new_data, [iris_data[i+1]],axis = 0)
print(iris_new_target)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1]


In [3]:
class logistic_regression:
    
    # производная в точке
    def derivative_log_function(self, X, Y, W, j):
        summ = 0
        for i in range(len(Y)):
            scal_prozv = np.dot(W,X[i])
            summ += -Y[i]*X[i,j]*np.exp(-Y[i]*scal_prozv)/(1+np.exp(-Y[i]*scal_prozv))
        return summ
    
    # метод градиентного спуска 
    def gradient_descent(self, X, Y, alpha, number_iter):
        
        W = np.zeros(len(X[0]))
    
        for k in range(number_iter):
            for j in range(len(X[0])):
                W[j]=  W[j] - alpha*self.derivative_log_function(X, Y, W, j)
        self.W = W
        return W
    
    # метод нестерова
    def nesterov_gradient_descent(self, X, Y, alpha, number_iter):
        
        W_nest = np.zeros(len(X[0]))
        v_x = np.zeros(len(X[0]))
        old_v = np.zeros(len(X[0]))
        r_x = 0.1
        
        for k in range(number_iter):
            for j in range(len(X[0])):
                old_v[j] = v_x[j]
                v_x[j] = r_x * v_x[j] - alpha * self.derivative_log_function(X, Y, W_nest, j)
                W_nest[j] = W_nest[j] - r_x * old_v[j] + (1 + r_x) * v_x[j]
                
        self.W_nest = W_nest
        return W_nest
    
    
    # метод RMSProp
    def rmsprop(self, X, Y, alpha, betta, e, number_iter):
        
        W_rms = np.zeros(len(X[0]))
        w_aver = np.zeros((len(X[0])))
        n_k = np.zeros((len(X[0])))
        
        for k in range(number_iter):         
            for j in range(len(X[0])):
                
                w_aver[j] = betta*w_aver[j] + (1-betta)*(self.derivative_log_function(X, Y, W_rms, j))**2
                n_k[j] = alpha/((w_aver[j]**(0.5) + e))
                W_rms[j]=  W_rms[j] - n_k[j]*self.derivative_log_function(X, Y, W_rms, j)
                           
        self.W_rms = W_rms
        return W_rms
    
    
    # тест
    def test(self,X_test,Y_test, W):
        Y_calculation = np.array([])
        for i in range(len(X_test[:,1])):
            scal_prozv = np.dot(W,X_test[i])
            if  round(1/(1+np.exp(-scal_prozv))) > 0.5:
                Y_calculation = np.append(Y_calculation, 1)
            else:
                Y_calculation = np.append(Y_calculation, -1)
                
        score = 0
        for i in range(len(X_test[:,1])):
            if int(Y_calculation[i]) == int(Y_test[i]):
                score = score + 1
        return score/ len(X_test[:,1])


    
    
# разделим на обучающую и тестовую выборку
X_train, X_test, Y_train, Y_test = train_test_split(iris_new_data, iris_new_target, test_size=0.3, random_state=42)


model_logic = logistic_regression()
# используем градиентный спуск для нахождения коэффициентов
model_logic.gradient_descent(X_train, Y_train, 0.01, 5000)
# используем метод Нестерова для нахождения коэффициентов
model_logic.nesterov_gradient_descent(X_train, Y_train, 0.01, 5000)
# используем метод RMSProp для нахождения коэффициентов
model_logic.rmsprop(X_train, Y_train, 0.9, 0.01, 0.0000001, 5000)



print("W = ",model_logic.W)
print("Wnest = ",model_logic.W_nest)
print("Wrmst = ",model_logic.W_rms)

print("Score =",model_logic.test(X_test, Y_test, model_logic.W))
print("Score_nest =",model_logic.test(X_test, Y_test, model_logic.W_nest))
print("Score_rmst =",model_logic.test(X_test, Y_test, model_logic.W_rms))



W =  [-1.98810285 -7.09847882  5.02217542  5.03022845]
Wnest =  [-1.98804784 -7.09832636  5.02196486  5.03037612]
Wrmst =  [-70.25509887   3.03178911  78.8905353   22.97396137]
Score = 0.8666666666666667
Score_nest = 0.8666666666666667
Score_rmst = 0.9
